In [1]:
from init import *
import time

In [2]:
model = Model()

In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5102235078811646],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.00028308675973676145,
  'n': 1024,
  'skewness': None,
  'stddev': 4.3687009565474e-07},
 'skewness': [0.039425794035196304],
 'stddev': [2.8429689407348633],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'error': ['../src/core/Model.cpp Model& Model::Run()',
  'Statistics model_version5(Cuda::StreamGuard&, KernelModelArgs&)',
  'Model execution timeout is reached.']}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

1.1 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.3274773955345154,
  0.3266710042953491,
  0.4951585829257965,
  0.5167135000228882,
  1.0978899002075195,
  1.1258209943771362],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.22861763834953308,
  'n': 1024,
  'skewness': None,
  'stddev': 0.00012036730186082423},
 'skewness': [-0.010412177070975304,
  -0.018496213480830193,
  0.010573307052254677,
  0.01739741489291191,
  -0.005134698003530502,
  0.045729756355285645],
 'stddev': [2.0162253379821777,
  1.9953268766403198,
  2.813796043395996,
  2.8328816890716553,
  4.458627700805664,
  4.462224960327148],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [8]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [9]:
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.327477,0.020162,2.016225,-0.010412
1,BrownianMotion,,1,1.0,2000,10000,0.326671,0.019953,1.995327,-0.018496
2,BrownianMotion,,0,2.0,4000,10000,0.495159,0.028138,2.813796,0.010573
3,BrownianMotion,,1,2.0,4000,10000,0.516714,0.028329,2.832882,0.017397
4,BrownianMotion,,0,5.0,10000,10000,1.097890,0.044586,4.458628,-0.005135
5,BrownianMotion,,1,5.0,10000,10000,1.125821,0.044622,4.462225,0.045730
